# Импорты и дефолтные функции и датафреймы

In [36]:
import requests

from pptx import Presentation
from pptx.util import Inches, Pt
import pandas as pd 

In [37]:
TOKEN_ID = '' # ← Поставь тут твой токен
API_V = '5.126'
PERIOD = ['2020-11-01', '2020-11-30'] # Задаём интервал сведения показателей ['от', 'до'], константа, вынести в начало

In [38]:
def folower_count_get(owner_id):
    try:
        rf = requests.get('https://api.vk.com/method/groups.getMembers', params={'group_id': owner_id,
                                                                                 'v': API_V,
                                                                                 'access_token': TOKEN_ID})
    except Exception as err:
        return err
    
    rf_response = rf.json()
    return rf_response['response']['count']

In [40]:
# Соберём количество подписчиков в пабликах, которые анализируем, для рассчёта ER Post и других метрик:
followers = {owner_id: folower_count_get(abs(owner_id)) for owner_id in pwpt_df['owner_id'].unique()}
followers

{-1: 382180}

In [41]:
# Читаем файл, который получили в блокноте с функциями парсинга
pwpt_df = pd.read_csv('vk_wall_get_1public.csv', header=None)
pwpt_df.columns=['N', 'id', 'from_id', 'owner_id', 'date', 'text', 'views', 'likes', 'reposts', 'comments']
pwpt_df['link'] = 'https://vk.com/wall' + pwpt_df['owner_id'].map(str) + '_' + pwpt_df['id'].map(str)

# Конвертируем дату
pwpt_df['date'] = pd.to_datetime(pwpt_df['date'], unit='s')

# Считаем и добавляем столбцы ER.v и ER.p
pwpt_df['ER.v'] = 100 * (pwpt_df['likes'] + pwpt_df['likes'] + pwpt_df['comments']) / pwpt_df['views']
pwpt_df['ER.p'] = 100 * (pwpt_df['likes'] + pwpt_df['likes'] + pwpt_df['comments']) / followers[-1]

pwpt_df

,N,id,from_id,owner_id,date,text,views,likes,reposts,comments,link,ER.v,ER.p
0,0,397203,-1,-1,2020-12-07 08:45:00,10 декабря в 19:00 собираемся на седьмом митап...,10288,47,0,40,https://vk.com/wall-1_397203,1.302488,0.035062
1,1,397107,-1,-1,2020-12-03 15:55:12,Прокачали messages.pin: отныне бот с правами а...,12201,131,3,69,https://vk.com/wall-1_397107,2.712892,0.086608
2,2,397065,-1,-1,2020-12-02 08:56:06,Теперь можно размещать свой проект сразу на дв...,6637,35,0,40,https://vk.com/wall-1_397065,1.657375,0.028782
3,3,396954,-1,-1,2020-11-13 13:34:17,"Делимся некоторыми изменениями, связанными с C...",13572,72,2,29,https://vk.com/wall-1_396954,1.274683,0.045267
4,4,396931,-1,-1,2020-11-12 12:48:35,Выведите заработок на новый уровень — на 130% ...,340186,376,2,13,https://vk.com/wall-1_396931,0.224877,0.200167
...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,176,240436,-1,-1,2011-02-05 11:54:32,Теперь openApi приложения тоже могут использов...,0,145,5,31,https://vk.com/wall-1_240436,inf,0.083992
177,177,238413,-1,-1,2011-02-04 15:40:53,Подключение виджетов в IFrame-приложениях http...,0,103,7,5,https://vk.com/wall-1_238413,inf,0.055210
178,178,236533,-1,-1,2011-02-03 18:17:06,Обновлён метод wall.post для публикации записе...,0,116,7,9,https://vk.com/wall-1_236533,inf,0.063059
179,179,55229,-1,-1,2010-11-08 20:13:17,Результаты второго этапа конкурса на разработк...,0,686,9,27,https://vk.com/wall-1_55229,inf,0.366058


In [42]:
owner_list = pwpt_df['owner_id'].unique()
owner_list

array([-1])

In [43]:
# Делаем срез датафрейма по заданному инетрвалу времени
srez_pwpt_df = pwpt_df[pwpt_df['date'].between(PERIOD[0], PERIOD[1], inclusive=True)]
srez_pwpt_df

,N,id,from_id,owner_id,date,text,views,likes,reposts,comments,link,ER.v,ER.p
3,3,396954,-1,-1,2020-11-13 13:34:17,"Делимся некоторыми изменениями, связанными с C...",13572,72,2,29,https://vk.com/wall-1_396954,1.274683,0.045267
4,4,396931,-1,-1,2020-11-12 12:48:35,Выведите заработок на новый уровень — на 130% ...,340186,376,2,13,https://vk.com/wall-1_396931,0.224877,0.200167
5,5,396877,-1,-1,2020-11-10 15:20:00,Открыли методы и события для работы с VK Donut...,15147,109,7,41,https://vk.com/wall-1_396877,1.709910,0.067769
6,6,396765,-1,-1,2020-11-02 16:01:01,Обновили метод [https://vk.com/dev/messages.se...,16891,131,1,54,https://vk.com/wall-1_396765,1.870819,0.082684


In [44]:
followers_sum = followers[-1] # пока руками зададим, потом циклом
views_sum = srez_pwpt_df['views'].sum()
likes_sum = srez_pwpt_df['likes'].sum()
reposts_sum = srez_pwpt_df['reposts'].sum()
comments_sum = srez_pwpt_df['comments'].sum()
ERv_mean = srez_pwpt_df['ER.v'].mean()
ERp_mean = srez_pwpt_df['ER.p'].mean()

# Формируем презентацию

In [79]:
prs = Presentation() # Инициализируем холст (воркбук если угодно), будет файлом в итоге

In [80]:
title_slide_layout = prs.slide_layouts[0]
title_slide = prs.slides.add_slide(title_slide_layout)
title = title_slide.shapes.title
subtitle = title_slide.placeholders[1]

title.text = "ОТЧЁТ ПО ПАБЛИКУ vk.com/public" + str(abs(owner_list[0]))
subtitle.text = "За период: с " + PERIOD[0] + '\n по ' + PERIOD[1]

In [81]:
bullet_slide_layout = prs.slide_layouts[1]

slide = prs.slides.add_slide(bullet_slide_layout)
shapes = slide.shapes

title_shape = shapes.title
body_shape = shapes.placeholders[1]

In [82]:
title_shape.text = "За период: с " + PERIOD[0] + '\n по ' + PERIOD[1]

tf = body_shape.text_frame
tf.text = "Сводные данные по паблику: "

p = tf.add_paragraph()
p.text = "В этом паблике всего подписчиков: " + str(followers_sum)
p.level = 1

p = tf.add_paragraph()
p.text = "В этом паблике всего просмотров: " + str(views_sum)
p.level = 1

p = tf.add_paragraph()
p.text = "В этом паблике всего лайков: " + str(likes_sum)
p.level = 1

p = tf.add_paragraph()
p.text = "В этом паблике всего репостов: " + str(reposts_sum)
p.level = 1

p = tf.add_paragraph()
p.text = "В этом паблике всего комментариев: " + str(comments_sum)
p.level = 1

In [83]:
bullet_slide_layout2 = prs.slide_layouts[1]

slide2 = prs.slides.add_slide(bullet_slide_layout2)
shapes2 = slide2.shapes

title_shape2 = shapes2.title
body_shape2 = shapes2.placeholders[1]

In [84]:
title_shape2.text = "За период: с " + PERIOD[0] + '\n по ' + PERIOD[1]

tf2 = body_shape2.text_frame
tf2.text = "Сводные данные Engagemen Rate: "

p2 = tf2.add_paragraph()
p2.text = "В этом паблике у постов средний ERv: " + str(ERv_mean)
p2.level = 1

p2 = tf2.add_paragraph()
p2.text = "В этом паблике у постов средний ERv: " + str(ERp_mean)
p2.level = 1

In [85]:
prs.save('демо отчёт преза 2.pptx')

""" Разобраться - почему съезжает размещение данных далеко влево за пределы слайда

left = top = width = height = Inches(1)
txBox = slide.shapes.add_textbox(left, top, width, height)
tf = txBox.text_frame

p = tf.add_paragraph()
p.text = "В этом паблике всего подписчиков: " + str(followers_sum)
p.font.size = Pt(20)

p = tf.add_paragraph()
p.text = "В этом паблике у постов суммарно просмотров: " + str(views_sum)
p.font.size = Pt(20)

p = tf.add_paragraph()
p.text = "В этом паблике у постов суммарно лайков: " + str(likes_sum)
p.font.size = Pt(20)

p = tf.add_paragraph()
p.text = "В этом паблике у постов суммарно репостов: " + str(reposts_sum)
p.font.size = Pt(20)

p = tf.add_paragraph()
p.text = "В этом паблике у постов суммарно комментариев: " + str(comments_sum)
p.font.size = Pt(20)

p = tf.add_paragraph()
p.text = "В этом паблике у постов средний ERv: " + str(ERv_mean)
p.font.size = Pt(20)

p = tf.add_paragraph()
p.text = "В этом паблике у постов средний ERp: " + str(ERp_mean)
p.font.size = Pt(20)
"""